In [ ]:
import requests
from bs4 import BeautifulSoup
from openpyxl import load_workbook

def query_word(word):
    url = f"https://cn.bing.com/dict/search?q={word}"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # parse pronunciation, part of speech, and meaning
        pronunciations = []
        for item in soup.find_all('div', class_='hd_p1_1'):
            pronunciations.append(item.get_text())
            # print(item.get_text())
        
        parts_of_speech = soup.find_all(class_='pos')
        meanings = soup.find_all(class_='def')

        meanings = []
        pos_and_defs = soup.find_all('div', class_='qdef')
        for pos_and_def in pos_and_defs:
            poses = [pos_.get_text() for pos_ in pos_and_def.find_all('span', class_='pos')]
            defs = [def_.get_text() for def_ in pos_and_def.find_all('span', class_='def')]
            meanings.extend([f'{pos_} {def_}'for pos_,def_ in zip(poses, defs)])
            print([f'{pos_} {def_}'for pos_,def_ in zip(poses, defs)]) 
        
        return {
            'pronunciations': pronunciations,
            'meanings': meanings
        }
    else:
        return None

def main():
    # load excel
    excel_file = 'words_list.xlsx'
    wb = load_workbook(excel_file)
    ws = wb.active

    # Iterate through each word
    for row_number, row in enumerate(sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=1, values_only=True), start=1):
        word = row[0]
            if word:
                # query word
                result = query_word(word)
                if result:
                    # write to excel
                    for i, meaning in enumerate(meanings, start=1):
                        sheet.cell(row=row_number, column=2).value = '\n'.join(pronunciations)
                        sheet.cell(row=row_number, column=2 + i).value = meaning

    # save Excel
    wb.save(excel_file)

if __name__ == "__main__":
    main()